In [5]:
path_to_video = "/mnt/750G/data/v2/video_540ss/"

In [6]:
from pathlib import Path

data_path = Path(path_to_video)

image_path_list = list(data_path.glob("*.mp4"))

In [8]:
image_path_list[1].__str__()

'/mnt/750G/data/v2/video_540ss/0091e812-f56f-4bd9-ad65-019fc97c1ffe.mp4'

In [14]:
import torch
import itertools
from torchvision.io import VideoReader

In [15]:
def example_read_video(video_path, start=0, end=None, read_video=True, read_audio=True):
    video_object = VideoReader(video_path, "video")
    if end is None:
        end = float("inf")
    if end < start:
        raise ValueError(
            "end time should be larger than start time, got "
            "start time={} and end time={}".format(start, end)
        )

    video_frames = torch.empty(0)
    video_pts = []
    if read_video:
        video_object.set_current_stream("video")
        frames = []
        for frame in itertools.takewhile(lambda x: x['pts'] <= end, video_object.seek(start)):
            frames.append(frame['data'])
            video_pts.append(frame['pts'])
        if len(frames) > 0:
            video_frames = torch.stack(frames, 0)

    return video_frames, video_pts, video_object.get_metadata()

In [16]:
vf, info, meta = example_read_video(image_path_list[1].__str__())
print(vf.size())

AttributeError: 'ImportError' object has no attribute 'open'